# 🧠 Knowledge Distillation: PyTorch Version (10-Class Classifiers)

This notebook performs training of:

1. A small CNN from scratch
2. Two pretrained CNNs (MobileNetV3 and VGG16) fine-tuned on CIFAR-10
3. Knowledge distillation from:
   - One teacher (MobileNetV3)
   - Two teachers (averaged predictions)
   - Two teachers (sequential training)

Includes accuracy comparison for all models.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.models import mobilenet_v3_small, vgg16
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

classes = train_dataset.classes


 16%|█▌        | 27.0M/170M [06:09<2:03:46, 19.3kB/s]

In [ ]:
class SmallCNN(nn.Module):
    def __init__(self):
        super(SmallCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout(0.25)
        self.fc1 = nn.Linear(32 * 16 * 16, 128)
        self.dropout2 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.dropout1(x)
        x = x.view(-1, 32 * 16 * 16)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        return self.fc2(x)


## 🧑‍🏫 Fine-tuning Pretrained Teachers (MobileNetV3, VGG16)

In [ ]:
def get_teacher_model(name='mobilenetv3'):
    if name == 'mobilenetv3':
        model = mobilenet_v3_small(pretrained=True)
    elif name == 'vgg16':
        model = vgg16(pretrained=True)
    else:
        raise ValueError("Invalid model name")

    for param in model.parameters():
        param.requires_grad = False

    if name == 'mobilenetv3':
        model.classifier[3] = nn.Linear(model.classifier[3].in_features, 10)
        for param in model.classifier[3].parameters():
            param.requires_grad = True
    else:  # vgg16
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, 10)
        for param in model.classifier[6].parameters():
            param.requires_grad = True

    return model.to(device)


## 🏋️ Training Function

In [ ]:
def train(model, loader, optimizer, criterion, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(loader):.4f}")


In [ ]:
def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return correct / total


## 🎓 Train and Evaluate Teacher Models

In [ ]:
teacher_mv3 = get_teacher_model("mobilenetv3")
teacher_vgg = get_teacher_model("vgg16")

opt_mv3 = torch.optim.Adam(teacher_mv3.parameters(), lr=1e-3)
opt_vgg = torch.optim.Adam(teacher_vgg.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

train(teacher_mv3, train_loader, opt_mv3, criterion, epochs=5)
train(teacher_vgg, train_loader, opt_vgg, criterion, epochs=5)

acc_mv3 = evaluate(teacher_mv3, test_loader)
acc_vgg = evaluate(teacher_vgg, test_loader)
print(f"MobileNetV3 Accuracy: {acc_mv3:.4f}")
print(f"VGG16 Accuracy: {acc_vgg:.4f}")


## 🔥 Knowledge Distillation Functions

In [ ]:
class Distiller(nn.Module):
    def __init__(self, student, teacher, temperature=3.0, alpha=0.5):
        super(Distiller, self).__init__()
        self.student = student
        self.teacher = teacher
        self.temperature = temperature
        self.alpha = alpha
        self.ce_loss = nn.CrossEntropyLoss()
        self.kl_loss = nn.KLDivLoss(reduction="batchmean")

    def forward(self, x):
        student_logits = self.student(x)
        with torch.no_grad():
            teacher_logits = self.teacher(x)
        return student_logits, teacher_logits

    def compute_loss(self, student_logits, teacher_logits, labels):
        soft_teacher = F.log_softmax(teacher_logits / self.temperature, dim=1)
        soft_student = F.log_softmax(student_logits / self.temperature, dim=1)
        distill_loss = self.kl_loss(soft_student, soft_teacher) * (self.temperature ** 2)
        ce = self.ce_loss(student_logits, labels)
        return self.alpha * ce + (1 - self.alpha) * distill_loss


In [ ]:
def train_distiller(distiller, loader, optimizer, epochs=5):
    distiller.train()
    for epoch in range(epochs):
        total_loss = 0
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            student_logits, teacher_logits = distiller(images)
            loss = distiller.compute_loss(student_logits, teacher_logits, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"[Distill Epoch {epoch+1}] Loss: {total_loss/len(loader):.4f}")


## 📘 Distillation from Single Teacher (MobileNetV3)

In [ ]:
student_single = SmallCNN().to(device)
distiller_single = Distiller(student_single, teacher_mv3, temperature=3.0, alpha=0.3)
opt_single = torch.optim.Adam(student_single.parameters(), lr=1e-3)

train_distiller(distiller_single, train_loader, opt_single, epochs=5)
acc_single = evaluate(student_single, test_loader)
print(f"Student (Distilled from MobileNetV3) Accuracy: {acc_single:.4f}")


## 📗 Distillation from Two Teachers (Average Prediction)

In [ ]:
class DistillerAverage(nn.Module):
    def __init__(self, student, teachers, temperature=3.0, alpha=0.5):
        super().__init__()
        self.student = student
        self.teachers = teachers
        self.temperature = temperature
        self.alpha = alpha
        self.ce_loss = nn.CrossEntropyLoss()
        self.kl_loss = nn.KLDivLoss(reduction="batchmean")

    def forward(self, x):
        student_logits = self.student(x)
        with torch.no_grad():
            teacher_logits = [t(x) for t in self.teachers]
            avg_teacher_logits = torch.mean(torch.stack(teacher_logits), dim=0)
        return student_logits, avg_teacher_logits

    def compute_loss(self, student_logits, teacher_logits, labels):
        soft_teacher = F.softmax(teacher_logits / self.temperature, dim=1)
        soft_student = F.log_softmax(student_logits / self.temperature, dim=1)
        distill_loss = self.kl_loss(soft_student, soft_teacher) * (self.temperature ** 2)
        ce = self.ce_loss(student_logits, labels)
        return self.alpha * ce + (1 - self.alpha) * distill_loss


In [ ]:
student_avg = SmallCNN().to(device)
distiller_avg = DistillerAverage(student_avg, [teacher_mv3, teacher_vgg], temperature=3.0, alpha=0.3)
opt_avg = torch.optim.Adam(student_avg.parameters(), lr=1e-3)

train_distiller(distiller_avg, train_loader, opt_avg, epochs=5)
acc_avg = evaluate(student_avg, test_loader)
print(f"Student (Distilled from both teachers - average) Accuracy: {acc_avg:.4f}")


## 📕 Distillation from Two Teachers (Sequential Training)

In [ ]:
student_seq = SmallCNN().to(device)

# Phase 1 - Teacher MobileNetV3
distiller_seq1 = Distiller(student_seq, teacher_mv3, temperature=3.0, alpha=0.3)
opt_seq1 = torch.optim.Adam(student_seq.parameters(), lr=1e-3)
train_distiller(distiller_seq1, train_loader, opt_seq1, epochs=3)

# Phase 2 - Teacher VGG16
distiller_seq2 = Distiller(student_seq, teacher_vgg, temperature=3.0, alpha=0.3)
opt_seq2 = torch.optim.Adam(student_seq.parameters(), lr=1e-3)
train_distiller(distiller_seq2, train_loader, opt_seq2, epochs=3)

acc_seq = evaluate(student_seq, test_loader)
print(f"Student (Distilled Sequentially) Accuracy: {acc_seq:.4f}")


## 📊 Final Accuracy Comparison of All Models

In [ ]:
labels = [
    'Teacher (MobileNetV3)', 
    'Teacher (VGG16)', 
    'Student (Scratch)', 
    'Student (Distilled by MobileNetV3)',
    'Student (Distilled by both teacher - average)',
    'Student (Distilled by both teacher - sequential)'
]

accuracies = [
    acc_mv3,
    acc_vgg,
    evaluate(SmallCNN().to(device), test_loader),  # dummy untrained student
    acc_single,
    acc_avg,
    acc_seq
]

# Plot
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
bars = plt.bar(labels, accuracies, color='skyblue', edgecolor='black')

for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 0.01, f"{yval:.4f}", ha='center', va='bottom', fontsize=10)

plt.ylim(0, 1.0)
plt.xticks(rotation=15)
plt.ylabel("Accuracy")
plt.title("Comparison of All Classifier Accuracies")
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


## 🔍 Confusion Matrix for Best Distilled Model

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns

# Get predictions from the best model (student_seq)
student_seq.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = student_seq(images)
        preds = outputs.argmax(dim=1).cpu().numpy()
        y_pred.extend(preds)
        y_true.extend(labels.numpy())

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=classes, yticklabels=classes, cmap='Blues')
plt.title("Confusion Matrix: Student (Distilled Sequentially)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.tight_layout()
plt.show()
